In [1]:
# 導入分詞套件
import jieba
from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever

# 使用 jieba 進行分詞
def tokenize(content):
    return ' '.join(jieba.cut(content))

# 建立文件並設置元數據，並使用 jieba 分詞
documents = [
    Document(
        content=tokenize("秦朝（公元前221年－公元前206年）是中國歷史上第一個統一的多民族中央集權制國家，開創了中國封建社會的先河。"),
        meta={"dynasty": "秦朝", "start_year": -221, "end_year": -206},
    ),
    Document(
        content=tokenize("漢朝（公元前202年－公元220年）分為西漢和東漢，是中國歷史上的一個重要朝代，開創了文景之治和漢武盛世。"),
        meta={"dynasty": "漢朝", "start_year": -202, "end_year": 220},
    ),
    Document(
        content=tokenize("唐朝（公元618年－公元907年）是中國歷史上的一個鼎盛時期，文化、經濟、軍事都達到了新的高峰。"),
        meta={"dynasty": "唐朝", "start_year": 618, "end_year": 907},
    ),
    Document(
        content=tokenize("宋朝（公元960年－公元1279年）分為北宋和南宋，兩宋時期的經濟和文化發展繁榮。"),
        meta={"dynasty": "宋朝", "start_year": 960, "end_year": 1279},
    ),
    Document(
        content=tokenize("元朝（公元1271年－公元1368年）是中國歷史上第一個由少數民族建立的全國統一王朝。"),
        meta={"dynasty": "元朝", "start_year": 1271, "end_year": 1368},
    ),
    Document(
        content=tokenize("明朝（公元1368年－公元1644年）是中國歷史上最後一個由漢族建立的大一統王朝。"),
        meta={"dynasty": "明朝", "start_year": 1368, "end_year": 1644},
    ),
    Document(
        content=tokenize("清朝（公元1644年－公元1912年）是中國歷史上的最後一個封建王朝，也是中國歷史上第二個由少數民族建立的統一王朝。"),
        meta={"dynasty": "清朝", "start_year": 1644, "end_year": 1912},
    ),
]

# 初始化內存文件儲存
document_store = InMemoryDocumentStore(bm25_algorithm="BM25Plus")
# 將文件寫入文件儲存
document_store.write_documents(documents=documents)
# 略作查看
document_store

/Users/samhsiao/Documents/PythonVenv/envHaystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/vm/_zft15050r7cfh47y6v5z4k40000gn/T/jieba.cache
Loading model cost 0.266 seconds.
Prefix dict has been built successfully.


In [3]:
# 查看文件的總數
document_count = document_store.count_documents()
print(f"儲存的文件數量：{document_count}")

儲存的文件數量：7


In [2]:
from haystack import Pipeline

# 初始化管道
pipeline = Pipeline()
# 添加內存 BM25 檢索器到管道
pipeline.add_component(
    instance=InMemoryBM25Retriever(document_store=document_store),
    name="retriever"
)

In [6]:
# 提問
query = "中國歷史上的朝代"
# 查詢也必須進行分詞
query_tokenized = tokenize(query)

# 不篩選，先輸出所有文件進行觀察
result = pipeline.run(
    data={"retriever": {"query": query_tokenized}}
)
# 輸出結果
print("不篩選的查詢結果")
print(result)
for doc in result["retriever"]["documents"]:
    print(
        f"{doc.meta['dynasty']} ({doc.meta['start_year']} - {doc.meta['end_year']}): {doc.content}"
    )

不篩選的查詢結果
{'retriever': {'documents': [Document(id=8d3d377c51f6d92373ee3317b6a0fc00e5569bcadcafcc901a6c0f229f561bc1, content: '漢朝 （ 公元前 202 年 － 公元 220 年 ） 分為 西漢 和 東漢 ， 是 中國 歷史 上 的 一個 重要 朝代 ， 開創 了 文景之治 和 漢武 盛世 。', meta: {'dynasty': '漢朝', 'start_year': -202, 'end_year': 220}, score: 3.933490489975213), Document(id=ed930a165645482c0f35c79dc6c3e6133c4755ccf0ff592960c52ddbd2f469d0, content: '清朝 （ 公元 1644 年 － 公元 1912 年 ） 是 中國 歷史 上 的 最後一個 封建王朝 ， 也 是 中國 歷史 上 第二 個 由 少數 民族 建立 的 統一 王朝 。', meta: {'dynasty': '清朝', 'start_year': 1644, 'end_year': 1912}, score: 2.6310913433509793), Document(id=ebd7b76b0a34d139291ed1dd94f9fc30b084c3c0489cb85c2b6922a96ee7ca46, content: '秦朝 （ 公元前 221 年 － 公元前 206 年 ） 是 中國 歷史 上 第一 個統 一 的 多 民族 中央 集權 制國家 ， 開創 了 中國 封建 社會 的 先河 。', meta: {'dynasty': '秦朝', 'start_year': -221, 'end_year': -206}, score: 2.532150132770046), Document(id=851f42bbaae277e8addd0d82c4d6c3824746f1f0f626316f942216218a680916, content: '明朝 （ 公元 1368 年 － 公元 1644 年 ） 是 中國 歷史 上 最後一個 由漢族 建立 的 大 一統 王朝 。', meta: {

In [7]:
# 進行元數據過濾
result = pipeline.run(
    data={
        "retriever": {
            "query": query_tokenized,
            "filters": {
                "start_year": {"$gt": 0}
            }
        }
    }
)

# 輸出結果
print("過濾條件：開始年份大於 0")
print(result)
for doc in result["retriever"]["documents"]:
    print(
        f"{doc.meta['dynasty']} ({doc.meta['start_year']} - {doc.meta['end_year']}): {doc.content}"
    )

過濾條件：開始年份大於 0
{'retriever': {'documents': [Document(id=ed930a165645482c0f35c79dc6c3e6133c4755ccf0ff592960c52ddbd2f469d0, content: '清朝 （ 公元 1644 年 － 公元 1912 年 ） 是 中國 歷史 上 的 最後一個 封建王朝 ， 也 是 中國 歷史 上 第二 個 由 少數 民族 建立 的 統一 王朝 。', meta: {'dynasty': '清朝', 'start_year': 1644, 'end_year': 1912}, score: 2.6310913433509793), Document(id=851f42bbaae277e8addd0d82c4d6c3824746f1f0f626316f942216218a680916, content: '明朝 （ 公元 1368 年 － 公元 1644 年 ） 是 中國 歷史 上 最後一個 由漢族 建立 的 大 一統 王朝 。', meta: {'dynasty': '明朝', 'start_year': 1368, 'end_year': 1644}, score: 2.521194406351849), Document(id=6c4fa1faead803e244dba169d81f561ef44e6093cc91c10a0b54c00a7e24ccd1, content: '元朝 （ 公元 1271 年 － 公元 1368 年 ） 是 中國 歷史 上 第一 個 由 少數 民族 建立 的 全國統 一 王朝 。', meta: {'dynasty': '元朝', 'start_year': 1271, 'end_year': 1368}, score: 2.5063213190672458), Document(id=24371144c44b73480ac04a07ed4e707c0208eb4f5b893ab714baac5c75727ac7, content: '唐朝 （ 公元 618 年 － 公元 907 年 ） 是 中國 歷史 上 的 一個 鼎盛 時期 ， 文化 、 經濟 、 軍事 都 達 到 了 新 的 高峰 。', meta: {'dynasty': '唐朝',

In [8]:
# 提問並進行複合條件過濾
result = pipeline.run(
    data={
        "retriever": {
            "query": query_tokenized,
            "filters": {
                "$and": [
                    {"start_year": {"$gt": 600}},
                    {"end_year": {"$lt": 1300}}
                ]
            }
        }
    }
)

# 輸出結果
print("複合條件過濾：開始年份大於 600 且結束年份小於 1300")
print(result)
for doc in result["retriever"]["documents"]:
    print(
        f"{doc.meta['dynasty']} ({doc.meta['start_year']} - {doc.meta['end_year']}): {doc.content}"
    )

複合條件過濾：開始年份大於 600 且結束年份小於 1300
{'retriever': {'documents': [Document(id=24371144c44b73480ac04a07ed4e707c0208eb4f5b893ab714baac5c75727ac7, content: '唐朝 （ 公元 618 年 － 公元 907 年 ） 是 中國 歷史 上 的 一個 鼎盛 時期 ， 文化 、 經濟 、 軍事 都 達 到 了 新 的 高峰 。', meta: {'dynasty': '唐朝', 'start_year': 618, 'end_year': 907}, score: 2.492438395707276), Document(id=f68631730f9b6bf859cadbccd0f4d4bf270e98d2d674b46d9d46fc1f3965808a, content: '宋朝 （ 公元 960 年 － 公元 1279 年 ） 分為 北宋 和 南宋 ， 兩宋 時期 的 經濟 和 文化 發展 繁榮 。', meta: {'dynasty': '宋朝', 'start_year': 960, 'end_year': 1279}, score: 2.089255163128161)]}}
唐朝 (618 - 907): 唐朝 （ 公元 618 年 － 公元 907 年 ） 是 中國 歷史 上 的 一個 鼎盛 時期 ， 文化 、 經濟 、 軍事 都 達 到 了 新 的 高峰 。
宋朝 (960 - 1279): 宋朝 （ 公元 960 年 － 公元 1279 年 ） 分為 北宋 和 南宋 ， 兩宋 時期 的 經濟 和 文化 發展 繁榮 。
